In [7]:
# Если нужно — раскомментируй строки ниже (и убедись, что есть интернет)
# !pip install transformers datasets accelerate sentencepiece

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset
import torch

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Загрузка модели и токенизатора
model_name = "Vikhrmodels/Vikhr-Gemma-2B-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Подготовка входного текста
input_text = "Напиши стихотворение о весне в России."

# Токенизация и генерация текста
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output = model.generate(input_ids, max_length=200, num_return_sequences=1, no_repeat_ngram_size=2)

# Декодирование и вывод результата
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.76it/s]


Напиши стихотворение о весне в России.


In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Vikhrmodels/Vikhr-Gemma-2B-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to("cuda")

prompt = "Напиши стихотворение о весне в России."

messages = [
    {"role": "user", "content": prompt}
]

# правильно формируем чатовый промпт
chat_prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)

output = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=True,
    top_p=0.9,
    temperature=0.8,
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:24<00:00,  8.09s/it]


user
Напиши стихотворение о весне в России.
model
Весна в России – это не просто сезон, это целый праздник природы. Его начало символизирует пробуждение и оживление природы после зимней спячки. Вот стихотворение, отражающее красоту и радость весеннего времени в России:

---

Весна в России придёт,
С улыбкой в сердце, словно в сказке.
Солнце встаёт, как в небесном саду,
И каждый цветок, словно крик, взорванный.

Тишину уносит ветер весенний,
Он шепчет о жизни, о мире, что ожил.
И деревья, словно танцоры в лесу,
Шепчут своим листьям весне, что она близка.

Птицы возвращаются с юга в ночи,
Они петь начали, как будто весна – это их песня.
И лучи солнца, как жемчужины, блестят,
На мокрой земле,


In [7]:
# небольшой срез, чтобы было быстро
from datasets import load_dataset
dataset = load_dataset("IlyaGusev/gazeta", split="test[:50]")
print(dataset[0])


c:\Users\reshg\OneDrive\Документы\baselines\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\reshg\.cache\huggingface\hub\datasets--IlyaGusev--gazeta. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to reg

{'text': 'На этих выходных в Берлине прошли крупные акции протеста против введенных для борьбы с коронавирусом ограничений. Демонстранты скандировали «Путин!» По словам депутата городской палаты представителей Гуннара Линдеманна («Альтернатива для Германии»), люди выкрикивали фамилию российского президента из уважения к нему. В комментарии РИА «Новости» немецкий политик отметил, что среди населения Германии Владимир Путин имеет хорошую репутацию. По его мнению, протестующие ранее пришли к российскому посольству, чтобы «привлечь внимание к условиям в Германии», надеясь, что Россия сможет оказать влияние на канцлера ФРГ Ангелу Меркель. «На мой взгляд, опасности для посольства России не возникло ни разу», — сказал депутат. Несмотря на то что протест оказался массовым, выступления носили «преимущественно мирный характер», уверен Линдеманн. По его словам, исключением стала только ситуацию у немецкого парламента. Там «несколько странных участников демонстрации попытались штурмовать бундестаг

In [1]:
from datasets import load_dataset

ds = load_dataset("IlyaGusev/gazeta")

# HuggingFace формат → pandas
train_df = ds["train"].to_pandas()
test_df  = ds["test"].to_pandas()

# сохраняем
train_df.to_csv("gazeta_train.csv", index=False)
test_df.to_csv("gazeta_test.csv", index=False)

print("Файлы сохранены:")
print("gazeta_train.csv →", train_df.shape)
print("gazeta_test.csv  →", test_df.shape)


c:\Users\reshg\OneDrive\Документы\baselines\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Файлы сохранены:
gazeta_train.csv → (60964, 5)
gazeta_test.csv  → (6793, 5)


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# предполагаю, что модель и токенайзер у тебя уже загружены:
# model_name = "Vikhrmodels/Vikhr-Gemma-2B-instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

def generate_summary(text: str, max_new_tokens: int = 80) -> str:
    """
    Генерируем краткое резюме статьи с помощью Vikhr-Gemma.
    """
    messages = [
        {
            "role": "user",
            "content": (
                "Сделай краткое абстрактное резюме следующей русской новостной статьи "
                "(1–3 предложения, выдели главное, не пиши лишних деталей):\n\n"
                + text
            ),
        }
    ]

    chat_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(chat_prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,     # для метрики лучше детерминированно
            num_beams=4,
            no_repeat_ngram_size=3,
        )

    # отрезаем префикс-промпт, чтобы не повторять вопрос
    gen_ids = outputs[0][inputs["input_ids"].shape[1]:]
    summary = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return summary.strip()


c:\Users\reshg\OneDrive\Документы\baselines\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'model' is not defined